In [1]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import scipy.stats as stats
import math
import seaborn as sns

In [2]:
df_transactions_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/" +
                                    "Thalia_transactions_ohne_vorname.csv")

df_history_wofirstname = pd.read_csv("C:/Users/felix.oechslein/Desktop/Daten, 13.12.22/Rohdaten/" +
                                    "Thalia_history_ohne_vorname.csv")

# Analyse 1: Test, ob pricetotal geeignete Größe für Ausreißer ist, wenn man nur Leute betrachtet, die mehr als eine Transaktion tätigten

## ----> NEIN

### df_t_grouped gibt Höhe und Anzahl an gesamten Transaktionen eines Kunden an

In [3]:
#Transaktions-Daten bereinigen

df_t_bereinigt = df_transactions_wofirstname[["iid", "key", "attrtype", "status", "pricetotal"]]
df_t_bereinigt = df_t_bereinigt.query('attrtype == "TI"')
df_t_bereinigt = df_t_bereinigt.query('status == "Buchung" or status == "Order"')
df_t_bereinigt = df_t_bereinigt.query('pricetotal >= 0.01')
#df_t_bereinigt = df_t2_bereinigt.sort_values("pricetotal")

In [4]:
#Daten gruppieren

list_t_grouped = df_t_bereinigt.groupby(["iid"])["pricetotal"].agg("sum")
df_t_grouped = list_t_grouped.to_frame()
df_t_grouped["AnzahlTrans"] = df_t_bereinigt.groupby(["iid"])["key"].agg("count")
#df_t_grouped = df_t_grouped.sort_values("pricetotal")
df_t_grouped.head()

,pricetotal,AnzahlTrans
iid,,
100002747,155.0,5
100009615,131.6,4
100009745,781.0,24
100011333,172.0,4
100011954,1206.1,63


In [5]:
# Computing IQR
#Q1 = df_t_grouped.query("AnzahlTrans > 1")['pricetotal'].quantile(0.25)
#Q3 = df_t_grouped.query("AnzahlTrans > 1")['pricetotal'].quantile(0.75)
Q1 = df_t_grouped['pricetotal'].quantile(0.25)
Q3 = df_t_grouped['pricetotal'].quantile(0.75)
IQR = Q3 - Q1
IQR_neu = 1.5 * IQR
print(IQR_neu)

158.70000000000002


In [6]:
# Computing IQR
Q1 = df_t_grouped.query("AnzahlTrans > 10")['pricetotal'].quantile(0.25)
Q3 = df_t_grouped.query("AnzahlTrans > 10")['pricetotal'].quantile(0.75)
IQR = Q3 - Q1
IQR_neu = 1.5 * IQR
print(IQR_neu)

894.375


# Analyse 2: Ergänzung der Transaktionsdaten mit FirstTrans und LastTrans Spalte

### df_t4 enthält FirstTrans, LastTrans, CostumerValue, AnzahlTrans

In [7]:
# Daten bereinigen

df_t_bereinigt2 = df_transactions_wofirstname[["iid", "key", "attrtype", "status", "pricetotal", "createdprovider"]]
df_t_bereinigt2 = df_t_bereinigt2.query('attrtype == "TI"')
df_t_bereinigt2 = df_t_bereinigt2.query('status == "Buchung" or status == "Order"')
df_t_bereinigt2 = df_t_bereinigt2.query('pricetotal >= 0.01')

# Daten-Datum

df_t_bereinigt2["CreatedProvider"] = pd.to_datetime(df_t_bereinigt2['createdprovider'], format='%Y%m%d%H%M%S')
df_t_bereinigt2['month_year'] = df_t_bereinigt2['CreatedProvider'].dt.to_period('M')

# Daten erste Transaktion, gespeichert in dt_3

list_t_first_trans = df_t_bereinigt2.groupby(["iid"])["createdprovider"].agg("min")
df_t_first_trans = list_t_first_trans.to_frame()
df_t_first_trans = df_t_first_trans.rename(columns = {"createdprovider" : "FirstTrans"})
df_t3 = pd.merge(df_t_bereinigt2, df_t_first_trans, on=["iid"])
df_t3["FirstTrans"] = pd.to_datetime(df_t3['FirstTrans'], format='%Y%m%d%H%M%S')

# Daten letzte Transaktion, gespeichert in dt_3

list_t_last_trans = df_t_bereinigt2.groupby(["iid"])["createdprovider"].agg("max")
df_t_last_trans = list_t_last_trans.to_frame()
df_t_last_trans = df_t_last_trans.rename(columns = {"createdprovider" : "LastTrans"})
df_t3 = pd.merge(df_t3, df_t_last_trans, on=["iid"])
df_t3["LastTrans"] = pd.to_datetime(df_t3['LastTrans'], format='%Y%m%d%H%M%S')

In [8]:
#Daten gruppieren

list_t3 = df_t3.groupby(["iid", "FirstTrans", "LastTrans"])["pricetotal"].agg("sum")
df_t3 = list_t3.to_frame()
df_t3 = df_t3.reset_index()
list_Anzahl_trans = df_t_bereinigt2.groupby(["iid"])["key"].agg("count")
df_Anzahl_trans = list_Anzahl_trans.to_frame()
df_Anzahl_trans = df_Anzahl_trans.rename(columns = {"key" : "AnzahlTrans"})
df_t4 = pd.merge(df_t3, df_Anzahl_trans, on=["iid"])
df_t4 = df_t4.rename(columns = {"pricetotal" : "CostumerValue"})
#df_t_grouped = df_t_grouped.sort_values("pricetotal")

In [9]:
df_t4.head()

,iid,FirstTrans,LastTrans,CostumerValue,AnzahlTrans
0,100002747,2022-02-03 12:44:22,2022-04-29 14:03:00,155.0,5
1,100009615,2018-09-10 09:52:00,2018-10-22 10:32:00,131.6,4
2,100009745,2021-10-29 11:52:50,2022-05-16 15:41:14,781.0,24
3,100011333,2022-05-27 10:24:33,2022-07-02 19:28:37,172.0,4
4,100011954,2010-03-24 22:07:00,2022-09-29 15:01:00,1206.1,63


# Analyse 3: Test, ob TIs pro CO geeignete Größe für Ausreißer ist

## -----> JA! 2/4 der größten Privatkunden sind immer noch enthalten (100405468, 100433284 ---  100445313, 100434361 nicht enthalten, obwohl wahrscheinlich Privatkunde)

## 18771193.56 / 26243302.33 = 71.53% des Gesamtumsatzes werden erfasst

### df_t_grouped_ohneAusreiser enthält nur Transaktionsdaten die weniger als 3.25 (Q3 + 1.5 * IQR des TIproOR Durchschnitts pro Kunden) TIs pro CO haben

In [10]:
#Transaktions-Daten bereinigen

df_t_OR = df_transactions_wofirstname[["iid", "key", "attrtype", "status", "pricetotal"]]
df_t_OR = df_t_OR.query('attrtype == "TI"')
df_t_OR = df_t_OR.query('status == "Buchung" or status == "Order"')
df_t_OR = df_t_OR.query('pricetotal >= 0.01')
#df_t_bereinigt = df_t2_bereinigt.sort_values("pricetotal")

In [11]:
df_t_OR_grouped = df_t_OR.groupby(["iid", "key"])["key"].agg("count")
df_t_OR_grouped = df_t_OR_grouped.to_frame()
df_t_OR_grouped = df_t_OR_grouped.rename(columns = {"key" : "AnzahlTIsProOrder"})
df_t_OR_grouped = df_t_OR_grouped.reset_index()

df_t_OR_grouped = df_t_OR_grouped.groupby("iid")["AnzahlTIsProOrder"].agg("mean")
df_t_OR_grouped = df_t_OR_grouped.to_frame()
df_t_OR_grouped = df_t_OR_grouped.rename(columns = {"AnzahlTIsProOrder" : "MeanTIperOR"})
df_t_OR_grouped = df_t_OR_grouped.reset_index()
df_t_OR_grouped.head()

,iid,MeanTIperOR
0,100002747,2.5
1,100009615,2.0
2,100009745,4.8
3,100011333,2.0
4,100011954,2.1


In [12]:
# Computing IQR
Q1 = df_t_OR_grouped["MeanTIperOR"].quantile(0.25)
Q3 = df_t_OR_grouped["MeanTIperOR"].quantile(0.75)
IQR = Q3 - Q1
IQR_neu = Q3 + 1.5 * IQR
print(IQR_neu)

3.25


In [13]:
#Merging iids der Nicht-Ausreiser mit Transaktionsdaten

df_Kunden_kein_Ausreißer_Anzahl_TIsProCO = df_t_OR_grouped.query("MeanTIperOR <= 3.25")
df_Kunden_kein_Ausreißer_Anzahl_TIsProCO_merged = pd.merge(df_t_OR, df_Kunden_kein_Ausreißer_Anzahl_TIsProCO, on=["iid"])

#Merging iids der Ausreisser mit Transaktionsdaten

df_Kunden_mit_Ausreißer_Anzahl_TIsProCO = df_t_OR_grouped.query("MeanTIperOR > 3.25")
df_Kunden_mit_Ausreißer_Anzahl_TIsProCO_merged = pd.merge(df_t_OR, df_Kunden_mit_Ausreißer_Anzahl_TIsProCO, on=["iid"])

In [14]:
#Daten gruppieren
#ohne Ausreisser
list_t_ohneAusreiser = df_Kunden_kein_Ausreißer_Anzahl_TIsProCO_merged.groupby(["iid"])["pricetotal"].agg("sum")
df_t_grouped_ohneAusreiser = list_t_ohneAusreiser.to_frame()
df_t_grouped_ohneAusreiser = df_t_grouped_ohneAusreiser.sort_values("pricetotal")
#mit Ausreisser
list_t_mitAusreiser = df_Kunden_mit_Ausreißer_Anzahl_TIsProCO_merged.groupby(["iid"])["pricetotal"].agg("sum")
df_t_grouped_mitAusreiser = list_t_mitAusreiser.to_frame()
df_t_grouped_mitAusreiser = df_t_grouped_mitAusreiser.sort_values("pricetotal")

In [15]:
#Messung wie viel vom Gesamtumsatz 26,24 Mio erfasst wird

print("Umsatz ohne Ausreisser:" + str(df_Kunden_kein_Ausreißer_Anzahl_TIsProCO_merged["pricetotal"].sum()))
print("Umsatz mit Ausreisser:" + str(df_Kunden_mit_Ausreißer_Anzahl_TIsProCO_merged["pricetotal"].sum()))

Umsatz ohne Ausreisser:18771193.560000002
Umsatz mit Ausreisser:7472108.7700000005


# Analyse 4: Betrachtung der Privatkunden

In [16]:
#Transaktions-Daten von Privatkunden mit History Daten mergen

df_t_ohneAusreißer_Betrachtung = df_t_grouped_ohneAusreiser
df_ohneAusreißer_Betrachtung_merged = pd.merge(df_t_ohneAusreißer_Betrachtung, df_history_wofirstname, on=["iid"])

In [17]:
df_ohneAusreißer_Betrachtung_merged.head()

,iid,pricetotal,mandator,id,attrtype,akey,text,creation
0,116868126,1.0,100000490,262840660,HY,261801331,|msg=CRM_Programm Spielzeit 22_23|subject=Ausb...,20220701193553
1,116868126,1.0,100000490,281712600,HY,280291785,|msg=Save the date_Lessingtage 2023|subject=Le...,20221101135410
2,116868126,1.0,100000490,158430668,NO,157893070,|msg=Save the Date_Lessingtage_2022|userAgent=...,20211103044451
3,116868126,1.0,100000490,158430952,NO,157893070,|msg=Save the Date_Lessingtage_2022|userAgent=...,20211103071745
4,116868126,1.0,100000490,158163331,HY,157893070,|msg=Save the Date_Lessingtage_2022|subject=Le...,20211101110323


In [18]:
df_t_ohneAusreißer_Betrachtung_merged["attrtype"].value_counts()

NameError: name 'df_t_ohneAusreißer_Betrachtung_merged' is not defined

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped =\
            df_t_ohneAusreißer_Betrachtung_merged.groupby(["iid", "pricetotal"])["attrtype"].value_counts().unstack()

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped.head()

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped =\
                df_t_ohneAusreißer_Betrachtung_merged_and_grouped.reset_index()
df_t_ohneAusreißer_Betrachtung_merged_and_grouped.head()

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped["Öffnungsrate"] =\
        df_t_ohneAusreißer_Betrachtung_merged_and_grouped["NO"] / df_t_ohneAusreißer_Betrachtung_merged_and_grouped["HY"]

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped.head()

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped.query("Öffnungsrate < 0.5 & pricetotal < 1000")\
                ["pricetotal"].plot(kind = "hist", bins = 100)

In [ ]:
df_t_ohneAusreißer_Betrachtung_merged_and_grouped.query("Öffnungsrate >= 0.5 & pricetotal < 1000")\
                ["pricetotal"].plot(kind = "hist", bins = 100)